# Aphrodisias analysis notebook


### Package imports

In [1]:
from pathlib import Path
import itertools

import pygeoprocessing
from osgeo import gdal, ogr
import statistics
import math
import numpy as np
import pandas as pd
import geopandas as gpd
import os
from shapely.geometry import Polygon
from matplotlib import pyplot as plt
import scipy
from scipy.stats import wilcoxon, pointbiserialr
import seaborn


gdal.UseExceptions()


### Defining input datasets

In [ ]:
gis_folder = Path(r"C:\Users\lizad\OneDrive\Desktop\Brown\Dissertation\GIS\Aphrodisias")

target_nodata = -9999.0

dem_raster_path = gis_folder / "processed_data/DEM_clip.tif"
dem_raster_info = pygeoprocessing.get_raster_info(str(dem_raster_path))

dem_treeheight_raster_path = gis_folder / "viewsheds/DEM_treeheight_combined.tif"
treeheight_raster_path = gis_folder / "viewsheds/treeheight_UTM_aligned.tif"

slope_raster_path = gis_folder / "slope_35N.tif"
tri_raster_path = gis_folder / "tri_35N.tif"
landcover_raster_path = gis_folder / "landcover_35N.tif"

church_vector_path = gis_folder / "aphrodisias_churches.gpkg"
church_name_field = "field_8"
city_center_vector_path = gis_folder / "center_35N.gpkg"
city_outline_vector_path = gis_folder / "city_outline.gpkg"

church_raster_path = gis_folder / "churches_35N.tif"
city_center_raster_path = gis_folder / "center_35N.tif"

multicriteria_lcps_vector_path = gis_folder / "final_lca/multicriteria_traditional.gpkg"
slope_lcps_vector_path = gis_folder / "final_lca/slope_traditional.gpkg"

# Intermediate files created within the script
## Aligned and clipped rasters
dem_raster_clip = gis_folder / "DEM_UTM_35N_clip.tif"
landcover_raster_clip = gis_folder / "landcover_35N_clip.tif"
slope_raster_clip = gis_folder / "slope_35N_clip.tif"
tri_raster_clip = gis_folder / "tri_35N_clip.tif"

## Landcover reclassified into resistence
landcover_reclass = gis_folder / "landcover_reclass.tif"

## Tobler Hiking rasters
tobler_surface_raster_white = gis_folder / "tobler_surface_white.tif"
tobler_rescale_raster_white = gis_folder / "tobler_rescale_white.tif"

tobler_surface_raster_mp = gis_folder / "tobler_surface_mp.tif"
tobler_rescale_raster_mp = gis_folder / "tobler_rescale_mp.tif"

## Combined cost surface rasters
cost_surface_white = gis_folder / "cost_surface_white.tif"
cost_surface_mp = gis_folder / "cost_surface_mp.tif"



# Parameters for circuitscape church analysis
connectivity_buffer_dist_m = 100
buffered_church_vector = gis_folder / f"churches_buffer_{connectivity_buffer_dist_m}m.gpkg"

circuitscape_church_raster = gis_folder / "circuitscape_church_source.tif"
circuitscape_city_raster = gis_folder / "circuitscape_city_ground.tif"



# Parameters for connectivity analysis and statistics
connectivity_raster = gis_folder / "circuitscape/omnidirectional_curmap_250516.tif"

sample_points_folder = gis_folder / "sample_points_analysis"
sample_area_vector_name  = "sample_area{}.gpkg"
sample_area_buffer_m = 500
sample_point_iterations = 100

buffered_church_name = "churches_buffer_{}m{}.gpkg"  # replace with connectivity_buffer_dist_m, results_suffix
buffered_church_zonal_stat_name = "churches_buffer_{}m_zonal_stats{}.gpkg"  # replace with connectivity_buffer_dist_m, results_suffix

current_var_structure = "omnidirectional_curmap_{}"
church_var = "church"

# TODO
church_designation_column = "church_designation"
church_designation_value = "suburban"
church_designation_results_suffix = "suburban"

## Pathnames for saving iteration outputs
sample_points_vector_name = "sample_points{}_{}.gpkg"  # replace with results_suffix, sample number

buffered_sample_points_vector_name = "buffered_sample_points_{}m{}_{}.gpkg"  # replace with connectivity_buffer_dist_m, results_suffix, sample number
buffered_sample_points_zonal_stats_vector_name = "zonal_stats_sample_points_{}m_zonal_stats{}_{}.gpkg"  # replace with connectivity_buffer_dist_m, results_suffix, sample number

combined_stats_vector_pathname =  "connectivity_statistics_sample_points{}_{}.gpkg"  # replace with results_suffix, sample number
histogram_name = "histogram_sample{}_{}_{}.png"  # replace with results_suffix, sample number, zonal_stat

combined_statistics_csv_name = "combined_statistical_results{}.csv"  # replace with results_suffix



# Parameters for viewshed analysis
church_height_m = 5
viewshed_distance_m = 10000

viewshed_results_folder = gis_folder / "viewsheds"
viewshed_results_folder.mkdir(exist_ok=True)

composite_viewshed = viewshed_results_folder / f"composite_viewshed.tif"

viewshed_treeheight_results_folder = gis_folder / "viewsheds/treeheight"
viewshed_treeheight_results_folder.mkdir(exist_ok=True)

composite_treeheight_viewshed = viewshed_treeheight_results_folder / "composite_treehight_viewshed.tif"

viewshed_sample_area = gis_folder / "viewsheds/cumulative_viewsheds_bbox.gpkg"
cumulative_viewshed_analysis_folder = gis_folder / "viewsheds/cumulative_viewshed_analysis"

viewshed_sample_point_iterations = 100
viewshed_sample_points_vector_name = cumulative_viewshed_analysis_folder / f"viewshed_sample_points_{viewshed_sample_point_iterations}.gpkg"
random_composite_viewshed = cumulative_viewshed_analysis_folder / "random_composite_viewshed.tif"

## Pathnames for saving iteration outputs
aligned_file_name = "aligned_{}"
viewshed_file_name = "viewshed_{}.tif"
treeheight_viewshed_file_name = "treehight_viewshed_{}.tif"



### Rasterizing Churches and city center to match the DEM

In [ ]:
# Create and burn church locations
if not church_raster_path.exists:
    pygeoprocessing.new_raster_from_base(
        str(dem_raster_path),
        str(church_raster_path),
        gdal.GDT_Byte,
        [0],
    )
    pygeoprocessing.rasterize(
    str(church_vector_path),
    str(church_raster_path),
    [1],
    )

# Create and burn city center
if not city_center_raster_path.exists:
    pygeoprocessing.new_raster_from_base(
        str(dem_raster_path),
        str(city_center_raster_path),
        gdal.GDT_Int16,
        [0],
    )
    pygeoprocessing.rasterize(
    str(city_center_vector_path),
    str(city_center_raster_path),
        [1],
    )

### Calculate slope and Terrain Ruggedness Index from DEM

In [ ]:
slope_gdal_ds = gdal.DEMProcessing(str(slope_raster_path), str(dem_raster_path), "slope")
del slope_gdal_ds

# pygeoprocessing.calculate_slope((str(dem_raster_path), 1), str(slope_raster_path))

tri_gdal_ds = gdal.DEMProcessing(str(tri_raster_path), str(dem_raster_path), "tri")
del tri_gdal_ds

### Normalize input datasets

In [ ]:
#Align and clip landscape data to DEM extent
dem_raster_info = pygeoprocessing.get_raster_info(str(dem_raster_path))

pygeoprocessing.align_and_resize_raster_stack(
    [str(dem_raster_path), str(landcover_raster_path), str(slope_raster_path), str(tri_raster_path)],
    [str(dem_raster_clip), str(landcover_raster_clip), str(slope_raster_clip), str(tri_raster_clip)],
    ["near", "near", "near", "near"],
    dem_raster_info["pixel_size"],
    dem_raster_info["bounding_box"],
    raster_align_index=0,
)

### Reclassify landcover into cost surface

In [ ]:
reclass_df = pd.read_csv("./data/esa_worldcover_classification.csv")
cost_value = "cost_value"

reclass_dict = reclass_df.set_index("lucode").to_dict()[cost_value]

pygeoprocessing.reclassify_raster(
    (str(landcover_raster_clip),1),
    reclass_dict,
    str(landcover_reclass),
    gdal.GDT_Float32,
    target_nodata,
)

### Calculate Tobler's hiking function (Tobler 1993)

In [ ]:
# tobler_surface_raster = gis_folder / "tobler_surface.tif"

# slope_raster_info = pygeoprocessing.get_raster_info(str(slope_raster_clip))
# slope_cell_resolution = statistics.mean([abs(x) for x in slope_raster_info["pixel_size"]])
# slope_nodata = slope_raster_info["nodata"][0]

# # def tobler_op(slope_array):
# #     result = (slope_cell_resolution/1000)/(6*np.exp(-3.5*np.abs(np.tan(slope_array*math.pi/180)+0.05)))
# #     return result
# # def tobler_op(slope):
# #     result = (slope_cell_resolution/1000)/(6*np.exp(-3.5*np.abs(slope+0.05)))
# #     return result

# def tobler_op(slope_array):
#     # Make an array of the same shape full of nodata
#     output = np.full(slope_array.shape, slope_nodata)

#     # Make a masking array to ignore all nodata areas in the original data
#     valid_mask = np.full(slope_array.shape, True)
#     valid_mask &= ~pygeoprocessing.array_equals_nodata(slope_array, slope_nodata)

#     output[valid_mask] = (slope_cell_resolution/1000)/(6*np.exp(-3.5*np.abs(np.tan(slope_array[valid_mask])+0.05)))
#     output[valid_mask] = (slope_cell_resolution/1000)/(6*np.exp(-3.5*np.abs(np.tan(slope_array[valid_mask]*math.pi/180)+0.05)))

#     return output

# pygeoprocessing.raster_calculator(
#     [(str(slope_raster_clip),1)], 
#     tobler_op,
#     str(tobler_surface_raster),
#     gdal.GDT_Float32,
#     target_nodata,
#     calc_raster_stats=True
# )


# # Define Tobler rescaling function
# def tobler_rescale_op(tobler_surface_array, upper_limit=0.1666667, lower_limit=0.0):
#     # Make an array of the same shape full of nodata
#     output = np.full(tobler_surface_array.shape, target_nodata)

#     # Make a masking array to ignore all nodata areas in the original data
#     valid_mask = np.full(tobler_surface_array.shape, True)
#     valid_mask &= ~pygeoprocessing.array_equals_nodata(tobler_surface_array, target_nodata)

#     # Calculate initial rescaling
#     output[valid_mask] = (tobler_surface_array[valid_mask] - lower_limit)/(upper_limit - lower_limit)

#     # Force values larger than the upper_limit to equal one
#     upper_limit_mask = (tobler_surface_array >= upper_limit) & valid_mask
#     output[upper_limit_mask] = 1

#     # Force values smaller than the lower_limit to equal zero
#     lower_limit_mask = (tobler_surface_array <= lower_limit) & valid_mask
#     output[lower_limit_mask] = 0

#     return output

# tobler_rescale_raster = gis_folder / "tobler_rescale.tif"

# # Rescaling Tobler's original function
# if tobler_rescale_raster.exists():
#     tobler_rescale_raster.unlink()
# pygeoprocessing.raster_calculator(
#     [(str(tobler_surface_raster),1)], 
#     tobler_rescale_op,
#     str(tobler_rescale_raster),
#     gdal.GDT_Float32,
#     target_nodata,
# )

### Calculate modified Tobler's hiking function (White 2015)

In [ ]:
slope_raster_info = pygeoprocessing.get_raster_info(str(slope_raster_clip))
slope_cell_resolution = statistics.mean([abs(x) for x in slope_raster_info["pixel_size"]])
slope_nodata = slope_raster_info["nodata"][0]

# def tobler_op(slope_array):
#     result = (slope_cell_resolution/1000)/(6*np.exp(-3.5*np.abs(np.tan(slope_array*math.pi/180)+0.05)))
#     return result
# def tobler_op(slope):
#     result = (slope_cell_resolution/1000)/(6*np.exp(-3.5*np.abs(slope+0.05)))
#     return result

def tobler_op_white(slope_array):
    # Make an array of the same shape full of nodata
    output = np.full(slope_array.shape, slope_nodata)

    # Make a masking array to ignore all nodata areas in the original data
    valid_mask = np.full(slope_array.shape, True)
    valid_mask &= ~pygeoprocessing.array_equals_nodata(slope_array, slope_nodata)

    output[valid_mask] = (slope_cell_resolution/1000)/(6*np.exp(-3.5*np.abs(np.tan(slope_array[valid_mask]*math.pi/180)+0.05)))

    return output

pygeoprocessing.raster_calculator(
    [(str(slope_raster_clip),1)], 
    tobler_op_white,
    str(tobler_surface_raster_white),
    gdal.GDT_Float32,
    target_nodata,
    calc_raster_stats=True
)

# Define Tobler rescaling function to convert to 0-1 index
def tobler_rescale_op(tobler_surface_array, upper_limit=0.1666667, lower_limit=0.0):
    # Make an array of the same shape full of nodata
    output = np.full(tobler_surface_array.shape, target_nodata)

    # Make a masking array to ignore all nodata areas in the original data
    valid_mask = np.full(tobler_surface_array.shape, True)
    valid_mask &= ~pygeoprocessing.array_equals_nodata(tobler_surface_array, target_nodata)

    # Calculate initial rescaling
    output[valid_mask] = (tobler_surface_array[valid_mask] - lower_limit)/(upper_limit - lower_limit)

    # Force values larger than the upper_limit to equal one
    upper_limit_mask = (tobler_surface_array >= upper_limit) & valid_mask
    output[upper_limit_mask] = 1

    # Force values smaller than the lower_limit to equal zero
    lower_limit_mask = (tobler_surface_array <= lower_limit) & valid_mask
    output[lower_limit_mask] = 0

    return output


# Rescaling Tobler's original function
if tobler_rescale_raster_white.exists():
    tobler_rescale_raster_white.unlink()
pygeoprocessing.raster_calculator(
    [(str(tobler_surface_raster_white),1)], 
    tobler_rescale_op,
    str(tobler_rescale_raster_white),
    gdal.GDT_Float32,
    target_nodata,
)

### Calculate modified Tobler's hiking function (Marquez-Perez 2017)

In [ ]:
slope_raster_info = pygeoprocessing.get_raster_info(str(slope_raster_clip))
slope_cell_resolution = statistics.mean([abs(x) for x in slope_raster_info["pixel_size"]])
slope_nodata = slope_raster_info["nodata"][0]

# def tobler_op(slope_array):
#     result = (slope_cell_resolution/1000)/(6*np.exp(-3.5*np.abs(np.tan(slope_array*math.pi/180)+0.05)))
#     return result
# def tobler_op(slope):
#     result = (slope_cell_resolution/1000)/(6*np.exp(-3.5*np.abs(slope+0.05)))
#     return result

def tobler_op_mp(slope_array):
    # Make an array of the same shape full of nodata
    output = np.full(slope_array.shape, slope_nodata)

    # Make a masking array to ignore all nodata areas in the original data
    valid_mask = np.full(slope_array.shape, True)
    valid_mask &= ~pygeoprocessing.array_equals_nodata(slope_array, slope_nodata)

    output[valid_mask] = (slope_cell_resolution/1000)/(4.8*np.exp(-5.3*np.abs((np.tan(slope_array[valid_mask]*math.pi/180)*0.7)+0.03)))

    return output

pygeoprocessing.raster_calculator(
    [(str(slope_raster_clip),1)], 
    tobler_op_mp,
    str(tobler_surface_raster_mp),
    gdal.GDT_Float32,
    target_nodata,
    calc_raster_stats=True
)


# Rescaling Tobler's original function
if tobler_rescale_raster_mp.exists():
    tobler_rescale_raster_mp.unlink()
pygeoprocessing.raster_calculator(
    [(str(tobler_surface_raster_mp),1)], 
    tobler_rescale_op,
    str(tobler_rescale_raster_mp),
    gdal.GDT_Float32,
    target_nodata,
)

### Create multicriteria cost surface (White 2015)

In [ ]:
# Get nodata values for toblers and lulc
landcover_raster_nodata = pygeoprocessing.get_raster_info(str(landcover_reclass))["nodata"][0]
tobler_raster_nodata = pygeoprocessing.get_raster_info(str(tobler_rescale_raster_white))["nodata"][0]


def weighted_average_op(tobler_array, lulc_cost_array, tobler_weight=0.8, lulc_cost_weight=0.2):
    # Make an array of the same shape full of nodata
    output = np.full(tobler_array.shape, target_nodata)

    # Make a masking array to ignore all nodata areas in the original data
    valid_mask = np.full(tobler_array.shape, True)
    valid_mask &= ~pygeoprocessing.array_equals_nodata(tobler_array, target_nodata)
    valid_mask &= ~pygeoprocessing.array_equals_nodata(lulc_cost_array, target_nodata)

    # Calculate weighted average
    output[valid_mask] = ((tobler_array[valid_mask]*tobler_weight) + (lulc_cost_array[valid_mask]*lulc_cost_weight)) / (tobler_weight+lulc_cost_weight)

    return output 

if cost_surface_white.exists():
    cost_surface_white.unlink()
pygeoprocessing.raster_calculator(
    [(str(tobler_rescale_raster_white),1),(str(landcover_reclass),1)],
    weighted_average_op,
    str(cost_surface_white),
    gdal.GDT_Float32,
    target_nodata,
)

### Create multicritera cost surface (Marquez-Perez 2017)

In [ ]:
# Get nodata values for toblers and lulc
landcover_raster_nodata = pygeoprocessing.get_raster_info(str(landcover_reclass))["nodata"][0]
tobler_raster_nodata = pygeoprocessing.get_raster_info(str(tobler_rescale_raster_mp))["nodata"][0]

if cost_surface_mp.exists():
    cost_surface_mp.unlink()
pygeoprocessing.raster_calculator(
    [(str(tobler_rescale_raster_mp),1),(str(landcover_reclass),1)],
    weighted_average_op,
    str(cost_surface_mp),
    gdal.GDT_Float32,
    target_nodata,
)

### Create 'Advanced' Circuitscape input files

In [ ]:

# Buffer church locations
church_location_gdf = gpd.read_file(church_vector_path)
church_location_gdf.geometry = church_location_gdf.buffer(connectivity_buffer_dist_m)
church_location_gdf.to_file(buffered_church_vector)

# Rasterize buffered church locations
pygeoprocessing.new_raster_from_base(
    str(dem_raster_path),
    str(circuitscape_church_raster),
    gdal.GDT_Int8,
    [100],
)

pygeoprocessing.rasterize(
    str(buffered_church_vector),
    str(circuitscape_church_raster),
    [10],
)

# Rasterize city
pygeoprocessing.new_raster_from_base(
    str(dem_raster_path),
    str(circuitscape_city_raster),
    gdal.GDT_Int8,
    [100],
)

pygeoprocessing.rasterize(
    str(city_outline_vector_path),
    str(circuitscape_city_raster),
    [0],
)

### Run tiled Circuitscape method

In [ ]:
# from pathlib import Path
# import circuitscape

# gis_folder = Path(r"C:\Users\lizad\OneDrive\Desktop\Brown\Dissertation\GIS")


# cost_surface_raster = gis_folder / "cost_surface.tif"
# workspace_path = gis_folder/ "circuitscape"
# seed_ini = gis_folder / "circuitscape/blank_circuitscape_ini.ini"
# # seed_ini = gis_folder / "circuitscape/blank_circuitscape_manual.ini"

# circuitscape.execute(cost_surface_raster, seed_ini, workspace_path, keep_intermediates=True)

### Analyzing connectivity by church

In [ ]:
# Define zonal statistics function
def connectivity_zonal_stats(connectivity_raster, vector, output_vector, join_gdf=None):
    zonal_stats_dict = pygeoprocessing.zonal_statistics(
        (str(connectivity_raster),1),
        str(vector),
    )
        
    zonal_stats_df = pd.DataFrame(zonal_stats_dict).T
    zonal_stats_df.index = zonal_stats_df.index -1
    zonal_stats_df["mean"] = zonal_stats_df["sum"] / zonal_stats_df["count"]
    zonal_stats_df = zonal_stats_df.rename(columns={c: f"omnidirectional_curmap_{c}" for c in zonal_stats_df.columns})

    vector_gdf = join_gdf if join_gdf is not None else gpd.read_file(vector)
    zonal_stats_gdf = vector_gdf.join(zonal_stats_df)
    zonal_stats_gdf.to_file(output_vector)


# Define full connectivity statistics function
def connectivity_statistics(
        church_location_gdf,
        connectivity_raster,
        sample_points_folder,

        sample_area_buffer_m=sample_area_buffer_m,
        sample_point_iterations=sample_point_iterations,

        results_suffix = '',
        connectivity_buffer_dist_m=connectivity_buffer_dist_m,
        ):
    
    # Define results suffix
    if results_suffix != "":
        results_suffix = f"_{results_suffix}"

    # Ensure results folder exists
    sample_points_folder.mkdir(exist_ok=True)

    # Get study area boundary for sample points analysis
    sample_area_gdf = church_location_gdf.dissolve().envelope.buffer(sample_area_buffer_m)
    sample_area_gdf.to_file(sample_points_folder / sample_area_vector_name.format(results_suffix))

    # Buffer churches and save file
    buffered_church_vector = sample_points_folder / buffered_church_name.format(connectivity_buffer_dist_m, results_suffix)
    church_location_gdf.geometry = church_location_gdf.buffer(connectivity_buffer_dist_m)
    church_location_gdf.to_file(buffered_church_vector)
    
    # Get connectivity raster info
    connectivity_raster_info = pygeoprocessing.get_raster_info(str(connectivity_raster))


    # bbox = connectivity_raster_info["bounding_box"]
    # sample_area_gdf = gpd.GeoDataFrame(pd.DataFrame(['bbox'], columns = ['geom']),
    #          crs = connectivity_raster_info["projection_wkt"],
    #          geometry = [Polygon([[bbox[0], bbox[1]],
    #         [bbox[2],bbox[1]],
    #         [bbox[2],bbox[3]],
    #         [bbox[0], bbox[3]]])
    # ])


    # Select number of sample points equal to the number of churches
    n_sample_points = len(church_location_gdf)

    # Run zonal stats on churches
    buffered_church_zonal_stats_vector = sample_points_folder / buffered_church_zonal_stat_name.format(connectivity_buffer_dist_m, results_suffix)
    connectivity_zonal_stats(connectivity_raster, buffered_church_vector, buffered_church_zonal_stats_vector, join_gdf=church_location_gdf)

    # Read in church zonal stats data and make "church" boolean column
    church_stats_gdf = gpd.read_file(buffered_church_zonal_stats_vector)
    church_stats_gdf[church_var] = True

    # Extract all raster connectivity values
    connectivity_array = pygeoprocessing.raster_to_numpy_array(str(connectivity_raster))
    connectivity_array[connectivity_array==connectivity_raster_info["nodata"]] = np.nan
    connectivity_array = connectivity_array[~np.isnan(connectivity_array)]

    # Iterate through sample point attempts, storing statistical outputs
    zonal_outputs = ["min", "max", "mean"]
    methods = ["pointbiserial", "wilcoxon"]
    values = ["statistic", "p_value"]
    data_dict = {"sample_iteration":[]}
    data_dict_key_format = "{}_{}_{}"
    data_dict.update({data_dict_key_format.format(zonal, method, value): [] for zonal, method, value in itertools.product(zonal_outputs,methods,values)})

    for i in range(sample_point_iterations):
        data_dict["sample_iteration"].append(i)

        # Name outputs based on the sample point number
        sample_points_vector = sample_points_folder / sample_points_vector_name.format(results_suffix,i)
        buffered_sample_points_vector = sample_points_folder / buffered_sample_points_vector_name.format(connectivity_buffer_dist_m, results_suffix,i)

        buffered_sample_points_zonal_stats_vector = sample_points_folder / buffered_sample_points_zonal_stats_vector_name.format(connectivity_buffer_dist_m, results_suffix,i)
        combined_stats_vector_path = sample_points_folder / combined_stats_vector_pathname.format(results_suffix,i)

        # Create sample points from study area bounds
        sampled_points = sample_area_gdf.sample_points(n_sample_points).explode()
        sampled_points.to_file(sample_points_vector)

        # Buffer sample points for zonal statistics
        sampled_points_buffer = sampled_points.buffer(connectivity_buffer_dist_m)
        sampled_points_buffer.to_file(buffered_sample_points_vector)

        # Run zonal stats on sample points
        connectivity_zonal_stats(
            connectivity_raster,
            buffered_sample_points_vector,
            buffered_sample_points_zonal_stats_vector,
            join_gdf=sampled_points)

        # Read in sample zonal stats data and make "church" boolean column
        sample_points_stats_gdf = gpd.read_file(buffered_sample_points_zonal_stats_vector)
        sample_points_stats_gdf[church_var] = False

        # Merge points into single dataset with church boolean field
        combined_stats_gdf = pd.concat([church_stats_gdf, sample_points_stats_gdf])
        combined_stats_gdf.to_file(combined_stats_vector_path)

        # Iterate through zonal stats type (min max mean) and calculate statistics on each
        for zonal in zonal_outputs:
            # stats_gdf = combined_stats_gdf[[current_var,church_var]].dropna()
            current_var = current_var_structure.format(zonal)

            stats_gdf = combined_stats_gdf[[current_var,church_var]]

            current_col = stats_gdf[current_var]
            church_col = stats_gdf[church_var]

            # Run correlation analysis on sample points vs churches
            pointbiserial_statistic, pointbiserial_p_value = pointbiserialr(church_col, current_col, nan_policy="omit")
            data_dict[data_dict_key_format.format(zonal,"pointbiserial","statistic")].append(pointbiserial_statistic)
            data_dict[data_dict_key_format.format(zonal,"pointbiserial","p_value")].append(pointbiserial_p_value)

            # Run wilcoxon thingy
            church_current_values = current_col[church_col]
            sample_current_values = current_col[~church_col]
            wilcoxon_statistic, wilcoxon_p_value = wilcoxon(church_current_values, sample_current_values, nan_policy="omit")
            data_dict[data_dict_key_format.format(zonal,"wilcoxon","statistic")].append(wilcoxon_statistic)
            data_dict[data_dict_key_format.format(zonal,"wilcoxon","p_value")].append(wilcoxon_p_value)

            # Create connectivity histogram
            fig, axs = plt.subplots(1, 1, figsize =(10, 7))

            # Plot raster connectivity histogram
            plt.hist(connectivity_array, color="skyblue", edgecolor="black") 
            plt.xlabel(f'Connectivity ({zonal})')
            plt.ylabel('Frequency')
            plt.title('Shart frequency') 
            axs.yaxis.set_ticks_position('none') 
            axs.axes.yaxis.set_ticklabels([]) 

            # # Plot church/sample locations as vertical lines
            # plt.vlines(x[y],ymin=0, ymax=3500000, colors="red")
            # plt.vlines(x[~y],ymin=0, ymax=3500000, colors="grey")

            # Create random array of values for scatterplot visualization (y axis of churches)
            rng = np.random.default_rng()
            sample_random_y = rng.uniform(100000,150000,len(current_col[~church_col]))
            church_random_y = rng.uniform(250000,350000,len(current_col[church_col]))

            # Plot church/sample locations as scatterplots
            plt.scatter(current_col[church_col],church_random_y, c="red", s=100, marker="o", edgecolors='black')
            plt.scatter(current_col[~church_col],sample_random_y, c="grey",s=100, marker="o", edgecolors='black')

            # Save figure
            histogram_figure = sample_points_folder / histogram_name.format(results_suffix.i,zonal)
            plt.savefig(histogram_figure, dpi=300)
            plt.close()
            # plt.show()

    statistics_df = pd.DataFrame(data_dict)
    statistics_df.to_csv(sample_points_folder / combined_statistics_csv_name.format(results_suffix))


# Read in church locations
church_location_gdf = gpd.read_file(church_vector_path)

# Run connectivity statistics on all churches
connectivity_statistics(church_location_gdf, connectivity_raster, sample_points_folder)

# Run connectivity statistics on subset of churches
church_location_gdf = church_location_gdf[church_location_gdf[church_designation_column] == church_designation_value]
connectivity_statistics(church_location_gdf, connectivity_raster, sample_points_folder, results_suffix=church_designation_results_suffix)




### Circuitscape analysis excluding suburban churches

In [ ]:
# Figure iteration

# Create connectivity histogram
fig, axs = plt.subplots(1, 1, figsize =(10, 7))

# Plot raster connectivity histogram 
plt.hist(connectivity_array, color="skyblue", edgecolor="black") 
plt.xlabel('Connectivity')
plt.ylabel('Frequency')
plt.title('Shart frequency') 
axs.yaxis.set_ticks_position('none') 
axs.axes.yaxis.set_ticklabels([]) 

# # Plot church/sample locations as vertical lines
# plt.vlines(x[y],ymin=0, ymax=3500000, colors="red")
# plt.vlines(x[~y],ymin=0, ymax=3500000, colors="grey")

# Create random array of values for scatterplot visualization (y axis of churches)
rng = np.random.default_rng()
sample_random_y = rng.uniform(100000,150000,len(current_col[~church_col]))
church_random_y = rng.uniform(250000,350000,len(current_col[church_col]))

# Plot church/sample locations as scatterplots
plt.scatter(current_col[church_col],church_random_y, c="red", s=100, marker="o", edgecolors='black')
plt.scatter(current_col[~church_col],sample_random_y, c="grey",s=100, marker="o", edgecolors='black')

# Save figure
plt.show()

from scipy.stats import shapiro
from scipy.stats import lognorm

from scipy.stats import wilcoxon

pointbiserialr(church_col, current_col)
wilcoxon()



### Viewshed analysis (DEM)


In [ ]:
# Open church point data
church_location_gdf = gpd.read_file(church_vector_path)

# TODO Iterate through churches
viewshed_rasters = []
for row in church_location_gdf.itertuples():
    church_x = row.geometry.coords[0][0]
    church_y = row.geometry.coords[0][1]
    output_viewshed = viewshed_results_folder / viewshed_file_name.format(getattr(row, church_name_field))
    viewshed_rasters.append(output_viewshed)

    # Open DEM dataset
    dem_raster_ds = gdal.Open(str(dem_raster_path))
    dem_raster_band = dem_raster_ds.GetRasterBand(1)

    dem_raster_info = pygeoprocessing.get_raster_info(str(dem_raster_path))

    hypotenuse_length = math.sqrt(
        (dem_raster_info["raster_size"][0]*dem_raster_info["pixel_size"][0])**2 +
        (dem_raster_info["raster_size"][1]*dem_raster_info["pixel_size"][1])**2
        )
    

    # https://gdal.org/en/stable/programs/gdal_viewshed.html
    ds = gdal.ViewshedGenerate(
        srcBand = dem_raster_band,
        driverName = 'GTiff',
        targetRasterName = str(output_viewshed),
        creationOptions = ["INTERLEAVE=BAND"],
        observerX = church_x,
        observerY = church_y,
        observerHeight = church_height_m,
        targetHeight = 2,
        visibleVal = 1,
        invisibleVal = 0,
        outOfRangeVal = 0,
        noDataVal = 0,
        dfCurvCoeff = 1 - 1/7,
        mode = gdal.GVOT_NORMAL,
        maxDistance = viewshed_distance_m)

    del ds


# Align all viewsheds
pygeoprocessing.align_and_resize_raster_stack(
    [str(r) for r in viewshed_rasters],
    [str(r.with_stem(aligned_file_name.format(r.name))) for r in viewshed_rasters],
    ["near"]*len(viewshed_rasters),
    dem_raster_info["pixel_size"],
    "union",
)

# Combine individual viewsheds into composite

def sum_op(*arrays):
    # Make an array of the same shape full of nodata
    output = np.full(arrays[0].shape, 0)

    # Calculate weighted average
    for r in arrays:
        output += r
    return output 

pygeoprocessing.raster_calculator(
    [(str(r.with_stem(aligned_file_name.format(r.name))),1) for r in viewshed_rasters],
    sum_op,
    str(composite_viewshed),
    gdal.GDT_Int8,
    0,
)


### Viewshed analysis (including tree height)

In [ ]:
# Open church point data
church_location_gdf = gpd.read_file(church_vector_path)

# TODO Iterate through churches
treeheight_viewshed_rasters = []
for row in church_location_gdf.itertuples():
    church_x = row.geometry.coords[0][0]
    church_y = row.geometry.coords[0][1]
    output_viewshed = viewshed_treeheight_results_folder / treeheight_viewshed_file_name.format(getattr(row, church_name_field))
    treeheight_viewshed_rasters.append(output_viewshed)

    # Open DEM dataset
    treeheight_raster_ds = gdal.Open(str(dem_treeheight_raster_path))
    treeheight_raster_band = treeheight_raster_ds.GetRasterBand(1)

    treeheight_raster_info = pygeoprocessing.get_raster_info(str(dem_treeheight_raster_path))

    hypotenuse_length = math.sqrt(
        (treeheight_raster_info["raster_size"][0]*treeheight_raster_info["pixel_size"][0])**2 +
        (treeheight_raster_info["raster_size"][1]*treeheight_raster_info["pixel_size"][1])**2
        )
    

    # https://gdal.org/en/stable/programs/gdal_viewshed.html
    ds = gdal.ViewshedGenerate(
        srcBand = treeheight_raster_band,
        driverName = 'GTiff',
        targetRasterName = str(output_viewshed),
        creationOptions = ["INTERLEAVE=BAND"],
        observerX = church_x,
        observerY = church_y,
        observerHeight = church_height_m,
        targetHeight = 2,
        visibleVal = 1,
        invisibleVal = 0,
        outOfRangeVal = 0,
        noDataVal = 0,
        dfCurvCoeff = 1 - 1/7,
        mode = gdal.GVOT_NORMAL,
        maxDistance = viewshed_distance_m)
    
    del ds

aligning_rasters = [treeheight_raster_path] + treeheight_viewshed_rasters
pygeoprocessing.align_and_resize_raster_stack(
    [str(r) for r in aligning_rasters],
    [str(r.with_stem(aligned_file_name.format(r.name))) for r in aligning_rasters],
    ["near"]*len(aligning_rasters),
    treeheight_raster_info["pixel_size"],
    "union",
)


# Combine individual viewsheds into composite

def sum_minus_trees_op(tree_canopy_array, *arrays, tree_height_cutoff=1.6):
    # Make an array of the same shape full of nodata
    output = np.full(arrays[0].shape, 0)

    # Calculate sum
    for r in arrays:
        output += r

    # Filter by areas with tree canopy less than the threshold
    tree_canopy_filter = tree_canopy_array <= tree_height_cutoff

    output *= tree_canopy_filter

    return output 

pygeoprocessing.raster_calculator(
    [(str(r.with_stem(aligned_file_name.format(r.name))),1) for r in aligning_rasters],
    sum_minus_trees_op,
    str(composite_treeheight_viewshed),
    gdal.GDT_Int8,
    0,
)


### Cumulative viewshed analysis

In [ ]:
#Read in bounding box for sample points
viewshed_sample_area_gdf = gpd.read_file(viewshed_sample_area)

# Create sample points in study area bounds
viewshed_sample_points = viewshed_sample_area_gdf.sample_points(viewshed_sample_point_iterations).explode()
viewshed_sample_points.to_file(viewshed_sample_points_vector_name)

viewshed_sample_points_gdf = gpd.read_file(viewshed_sample_points_vector_name)

random_viewshed_rasters = []
for i, row in enumerate(viewshed_sample_points_gdf.itertuples()):
    point_x = row.geometry.coords[0][0]
    point_y = row.geometry.coords[0][1]
    output_viewshed = cumulative_viewshed_analysis_folder / viewshed_file_name.format(i)
    random_viewshed_rasters.append(output_viewshed)

     # Open DEM dataset
    treeheight_raster_ds = gdal.Open(str(dem_treeheight_raster_path))
    treeheight_raster_band = treeheight_raster_ds.GetRasterBand(1)
    treeheight_raster_info = pygeoprocessing.get_raster_info(str(dem_treeheight_raster_path))

    #Run viewshed analysis on each sample point
       # https://gdal.org/en/stable/programs/gdal_viewshed.html
    ds = gdal.ViewshedGenerate(
        srcBand = treeheight_raster_band,
        driverName = 'GTiff',
        targetRasterName = str(output_viewshed),
        creationOptions = ["INTERLEAVE=BAND"],
        observerX = point_x,
        observerY = point_y,
        observerHeight = 1.6,
        targetHeight = 2,
        visibleVal = 1,
        invisibleVal = 0,
        outOfRangeVal = 0,
        noDataVal = 0,
        dfCurvCoeff = 1 - 1/7,
        mode = gdal.GVOT_NORMAL,
        maxDistance = viewshed_distance_m)
    
    del ds

# Combine individual viewsheds into composite
def sum_minus_trees_op(tree_canopy_array, *arrays, tree_height_cutoff=1.6):
    # Make an array of the same shape full of nodata
    output = np.full(arrays[0].shape, 0)

    # Calculate sum
    for r in arrays:
        output += r

    # Filter by areas with tree canopy less than the threshold
    tree_canopy_filter = tree_canopy_array <= tree_height_cutoff

    output *= tree_canopy_filter

    return output 

aligning_rasters = [treeheight_raster_path] + random_viewshed_rasters
pygeoprocessing.align_and_resize_raster_stack(
    [str(r) for r in aligning_rasters],
    [str(r.with_stem(aligned_file_name.format(r.name))) for r in aligning_rasters],
    ["near"]*len(aligning_rasters),
    treeheight_raster_info["pixel_size"],
    "union",
)

pygeoprocessing.raster_calculator(
    [(str(r.with_stem(aligned_file_name.format(r.name))),1) for r in aligning_rasters],
    sum_minus_trees_op,
    str(random_composite_viewshed),
    gdal.GDT_Int8,
    0,
)


### Viewsheds and LCP analysis

In [ ]:
# Open church point data
church_location_gdf = gpd.read_file(church_vector_path, engine="pyogrio", fid_as_index=True)
multicriteria_lcps_gdf = gpd.read_file(multicriteria_lcps_vector_path)
slope_lcps_gdf = gpd.read_file(slope_lcps_vector_path)

# Iterate through churches
treeheight_viewshed_rasters = []
for row in church_location_gdf.itertuples():
    output_viewshed = viewshed_treeheight_results_folder / treeheight_viewshed_file_name.format(getattr(row, church_name_field))
    if not output_viewshed.exists():
        raise ValueError("No viewshed for this church has been found")
    multicriteria_lcp = multicriteria_lcps_gdf[multicriteria_lcps_gdf["end point id"]==row.Index]
    slope_lcp = slope_lcps_gdf[slope_lcps_gdf["end point id"]==row.Index]
    output_viewshed_ds = gdal.Open(str(output_viewshed))
    output_viewshed_band = output_viewshed_ds.GetRasterBand(1)
    viewshed_crs = output_viewshed_ds.GetSpatialRef()
    output_viewshed_polygon = gis_folder / f"viewsheds/LCP_analysis/church_polygonized_viewshed_{getattr(row, church_name_field)}.gpkg"
    if output_viewshed_polygon.exists():
        output_viewshed_polygon.unlink()
    
    # Create output vector layer using GDAL, which is very dumb
    drv = ogr.GetDriverByName("GPKG")
    viewshed_field = 'viewshed'
    ds = drv.CreateDataSource(str(output_viewshed_polygon))
    layer = ds.CreateLayer(output_viewshed_polygon.stem, srs = viewshed_crs)
    field_defn = ogr.FieldDefn(viewshed_field, ogr.OFTInteger)
    layer.CreateField(field_defn)
    gdal.Polygonize(output_viewshed_band, None, layer, 0, [], callback=None)
    # Subset to no viewshed (i.e. value of 0) and delete
    layer.SetAttributeFilter(f"{viewshed_field} = 0")
    for feature in layer:
        fid = feature.GetFID()
        layer.DeleteFeature(int(fid))

    del ds
    del layer

    # Calculate intersection between viewshed and LCP
    output_viewshed_gdf = gpd.read_file(output_viewshed_polygon)
    output_viewshed_gdf = output_viewshed_gdf.dissolve()

    test = slope_lcp.overlay(output_viewshed_gdf)



In [ ]:
#Calculate intersection between viewshed and LCP
output_viewshed_gdf = gpd.read_file(output_viewshed_polygon)
output_viewshed_gdf = output_viewshed_gdf.dissolve()

import matplotlib.pyplot as plt

slope_lcp = slope_lcps_gdf[slope_lcps_gdf["end point id"]==19]


test = slope_lcp.overlay(output_viewshed_gdf)

# output_viewshed_polygon
# slope_lcp
test.length
slope_lcp.length.values[0]

test.length.values[0] / slope_lcp.length.values[0]


# # Plotting for testing
# fig, ax = plt.subplots()
# output_viewshed_gdf.plot(ax=ax)
# slope_lcps_gdf.plot(ax=ax)
# slope_lcp.plot(ax=ax, color="red")
# test.plot(ax=ax, color="red")



np.float64(0.020077582963266657)

In [4]:
# Dumb tree data download CLI commands for dumb trees

### THIS IS ALL IN THE TERMINAL. SCARY

### Go to folder where you will download things
# cd FOLDER_YOU_WILL_DOWNLOAD_TO

### Scan AWS resource folder (SKIP IF YOU WANT)
# aws s3 ls --no-sign-request s3://dataforgood-fb-data/forests/v1/alsgedi_global_v6_float/ --human-readable

### Download tile file for IDing which tiles to actually download (SKIP SINCE WE ALREADY DOWNLOADED IT)
# aws s3 cp --no-sign-request s3://dataforgood-fb-data/forests/v1/alsgedi_global_v6_float/tiles.geojson tiles.geojson

### Download an individual tile 
# aws s3 cp --no-sign-request s3://dataforgood-fb-data/forests/v1/alsgedi_global_v6_float/chm/[NUMBER OF TILE].tif tree_height_[NUMBER OF TILE].tif

# tiles
aphrodisias = [122101202, 122101203, 122101220, 122101221]